<a href="https://colab.research.google.com/github/A1ienSword/Pattern-recognition-labs/blob/main/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_3_%D0%9A%D0%BE%D1%81%D1%82%D0%B8%D1%86%D1%8B%D0%BD_%D0%92%D0%92_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа №3. Метод опорных векторов.
1. Написать программу, реализующую классификацию объектов, заданных признаковым
описанием, на основании метода опорных векторов (SVM) для случая с «мягким
зазором» без использования kernel trick.
2. Продемонстрировать работу программы на подготовленном наборе данных.
3. Функция, реализующая алгоритм SVM, должна быть реализована студентом
самостоятельно. Допускается использование готовых примитивов из стандартных
или сторонних библиотек для выполнения базовых (в т.ч. векторизованных) операций
(сложение, умножение, возведение в степень, вычисление среднего значения и т.п.).
4. Требования к набору данных: не менее 30 объектов, не менее 2 и не более 3 классов, не менее 3 и не более 7 атрибутов у объекта. Можно взять предложенные наборы данных «Ирисы» либо «Фрукты и овощи».
5. При выполнении задания рекомендуется использовать язык программирования Python или среду Octave.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
# Загрузка данных
data = pd.read_csv('iris.csv.gz')

In [3]:
# Разделение данных на признаки (X) и метки классов (y)
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [4]:
# Преобразование меток в числовые
le = LabelEncoder()
y = le.fit_transform(y)
# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=23)

In [5]:
# Нормализация данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
def train_svm(X, y, C=1.0, learning_rate=0.001, iterations=1000, verbose=False):
    """
    Обучение бинарного классификатора SVM с мягким зазором.

    Параметры:
    X - признаки обучающей выборки
    y - метки классов (1 и -1)
    C - параметр регуляризации
    learning_rate - скорость обучения
    iterations - количество итераций
    verbose - вывод потерь в процессе обучения
    """
    n_samples, n_features = X.shape
    w = np.zeros(n_features)
    b = 0.0

    for it in range(iterations):
        scores = np.dot(X, w) + b
        y_pred = y * scores
        hinge_loss = np.maximum(0, 1 - y_pred)
        loss = 0.5 * np.dot(w, w) + C * np.sum(hinge_loss)

        mask = hinge_loss > 0

        if np.sum(mask) == 0:
            grad_w = w
            grad_b = 0
        else:
            grad_w = w - C * np.dot(X[mask].T, y[mask])
            grad_b = -C * np.sum(y[mask])

        w -= learning_rate * grad_w
        b -= learning_rate * grad_b

        if verbose and (it % 100 == 0):
            print(f'Итерация {it}, Потери: {loss:.4f}')

    return w, b

In [7]:
# Функция предсказания классов
def predict(X):
    decision_scores = np.zeros((X.shape[0], len(classes)))
    for i, (w, b) in enumerate(classifiers):
        scores = np.dot(X, w) + b
        decision_scores[:, i] = scores
    return np.argmax(decision_scores, axis=1)

In [8]:
# Обучение мультиклассового классификатора по схеме OvR
classes = np.unique(y)
classifiers = []
for cls in classes:
    # Создание бинарных меток для текущего класса
    y_binary = np.where(y_train == cls, 1, -1)
    # Обучение SVM
    w, b = train_svm(X_train_scaled, y_binary, C=1.0, learning_rate=0.01, iterations=1000)
    classifiers.append((w, b))

In [9]:
# Оценка точности на тестовой выборке
y_pred = predict(X_test_scaled)
accuracy = np.mean(y_pred == y_test)
print(f'Точность на тестовой выборке: {accuracy:.2f}')

Точность на тестовой выборке: 1.00
